In [1]:
import sys
import codecs
import numpy as np
import pygimli as pg

try:
    import pybert as pb
except ImportError:
    sys.stderr.write('''ERROR: cannot import the library 'pybert'. ''' +
                     '''Ensure that pybert is in your PYTHONPATH ''')
    sys.exit(1)


In [2]:
def readAsDictionary(content, token=None, sep=None):
    """Read list of strings from a file as column separated dictionary.

        e.g.
        token1 token2 token3 token4
        va1    va2    val3   val4
        va1    va2    val3   val4
        va1    va2    val3   val4

    Parameters
    ----------
    content: [string]
        List of strings read from file:
        e.g.
        with open(filename, 'r') as fi:
            content = fi.readlines()
        fi.close()
    token: [string]
        If given the tokens will be the keys of the resulting dictionary.
        When token is None, tokens will be the first row values.
        When token is a empty list, the tokens will be autonamed to
        'col' + str(ColNumber)
    ret: dictionary
        Dictionary of all data
    """

    data = dict()

    if token is None:
        header = content[0].splitlines()[0].split(sep)
        token = []

        for i, tok in enumerate(header):
            tok = tok.lstrip()
            token.append(tok)

    for i, row in enumerate(content[1:]):
        vals = row.splitlines()[0].split(sep)
        for j, v in enumerate(vals):
            v = v.replace(',', '.')

            if len(token) < j+1:
                token.append('col' + str(j))
            if token[j] not in data:
                data[token[j]] = [None] * (len(content)-1)
            try:
                data[token[j]][i] = float(v)
            except:
                if len(v) == 1 and v[0] == '-':
                    v = 0.0
                data[token[j]][i] = v

    return data

In [3]:
def import4PointLight(filename, verbose=False):
    # filename=('2022-05-15_09-00-00.tx0')
    # verbose=True
    """Import Lippmann 4-point light instrument data (*.tx0)"""
    known_tokens = ['A', 'B', 'M', 'N', 'U', 'I', 'rho', 'phi', 'dU', 'dU90']
    data = pb.DataContainerERT()
    nel = 0
    datnr, toknr = [], []

#    with open(filename, 'r') as fi:
    with codecs.open(filename, 'r', encoding='utf8', errors='replace') as fi:
        content = fi.readlines()
#    fi.close()  # not needed using with

    dataSect = -3

#    tok_units = {'err': 0.01, }  # to be later used

    for i, line in enumerate(content):
        if verbose:
            pass
#            print(i, line)

        line = line.rstrip('\n').replace(',', '.')
        if (line.find('* Electrode last num') >= 0):
            nel = int(line.split()[-1])  # number of electrodes
            if verbose:
                print(nel, "electrodes")
            for n in range(nel):
                # create dummy sensors in case of Roll-along files where first
                # sensors are missing
                data.createSensor(pg.RVector3(-1000.+n, 0., 0.))

        if line.find('* Count') == 0:
            nData = int(line.split()[-1])  # number of data

        if (line.find('* Electrode [') == 0):
            sxyz = line.split()[-3:]
            if sxyz[-1] == 'X':
                sxyz = line.split()[-4:-1]
            pos = pg.RVector3(float(sxyz[0]), float(sxyz[1]), float(sxyz[2]))
            data.setSensorPosition(int(line[13:16])-1, pos)

        if (line.find('* num') == 0):
            if 1:
                d = readAsDictionary(content[i+1:], content[i].split()[1:])
                if verbose:
                    print("Token:", d.keys())

#                 usual token are
# ['num', 'A', 'B', 'M', 'N', 'I', 'U', 'dU', 'U90', 'dU90', 'rho', 'phi', 'f',
# 'n', 'nAB', 'Profile', 'Spread', 'PseudoZ', 'X', 'Y', 'Z', 'Date', 'Time',
# 'U(Tx)']
                if d['num'][-1] != len(d['num']):
                    print(d['num'][-1], len(d['num']))
                    raise Exception('Insufficient data found!')

                data.resize(len(d['num']))
                for i in range(data.size()):
                    data.createFourPointData(
                        i, int(d['A'][i]-1), int(d['B'][i]-1),
                        int(d['M'][i]-1), int(d['N'][i]-1))

                data.set('i', np.array(d['I'])/1000.0)
                data.set('u', np.array(d['U'])/1000.0)
                data.set('ip', np.array(d['phi']))
                data.set('rhoa', np.array(d['rho']))
                data.set('err', np.array(d['dU']))

            else:

                # evaluate tokens (physical fields) in file
                for nr, tok in enumerate(line[1:].split()):
                    if (known_tokens.count(tok)):
                        datnr.append(nr)
                        toknr.append(known_tokens.index(tok))

                known_tokens[6:10] = ['rhoa', 'ip', 'err', 'iperr']  # to BERT
                if verbose:
                    print(datnr, toknr, [known_tokens[t] for t in toknr])
                dataSect = -2  # prepare to be in data section 2 lines later
                # unit line should be evaluated to gain multiplicator
                data.resize(nData)  # initialize vectors with appropriate size

            if dataSect >= 0:  # read actual data if in data section
                sabmn = line.split()

                if dataSect >= nData:
                    data.resize(dataSect+1)
                # create electrode array
                data.createFourPointData(dataSect,
                                         int(sabmn[1])-1, int(sabmn[2])-1,
                                         int(sabmn[3])-1, int(sabmn[4])-1)

                for i in range(4, len(toknr)):  # all tokens except ABMN
                    itok = known_tokens[toknr[i]].lower()
                    try:
                        ff = float(sabmn[datnr[i]])
                        if itok.lower() == 'err':
                            ff = ff / 100
                        if itok.lower() == 'i':
                            ff = ff / 1000.
                        if itok.lower() == 'u':
                            ff = ff / 1000.

                        data(itok)[dataSect] = ff
                    except IndexError:
                        pass

            if dataSect >= -2:
                dataSect += 1  # so that unit line will not be read

#    savestr = ''
#    for i in range(len(toknr)):
#        savestr += known_tokens[toknr[i]].lower() + ' '

#    data.setInputFormatString(savestr)

    if data.size() == 0:
        raise BaseException('No Data found in import4PointLight')

    # data.set('rhoa', pg.abs(data('rhoa')))
    # data.checkDataValidity()

    return data

In [4]:
def exportData(data, filename, fmt='Gimli', verbose=False):
    """ Export datafile:

        supported: syscal -- syscal sequence
    """
    if fmt.lower() == 'gimli':
        data.save(filename)
    elif 'resecs' in fmt.lower():
        exportResecsAsciiFile(data, filename, verbose)
    elif 'res2dinv' in fmt.lower():
        exportRes2dInv(data, filename, verbose)
    elif 'syscal' in fmt.lower():
        writeSyscalSequence(data, filename, verbose)
    else:
        raise Exception("exportData format " + str(fmt) +
                        " not yet implemented. Please contact support.")

In [9]:
folder= r"Raw" # Update this to reflect the folder lcoation
os.chdir(folder)
for filename in os.listdir(folder):
    if filename.endswith('.tx0'):
        # input
        input_file_path = os.path.join(folder, filename)
        
        # output
        output_file_name = filename.replace('.tx0', '.txt')
        
        # outputpath
        output_file_path = os.path.join(folder, output_file_name)
        
        # probe and gap
        fixed_data = ["48# Number of electrodes", "# x z"] + [f"{i}     0" for i in range(48)] + ["909# Number of data"]
        
        data = []
        with open(input_file_path, 'r') as input_file:
            lines = input_file.readlines()
            data = [line.strip().split() for line in lines[240:]]
        
        selected_data = []
        for row in data:
            row_transformed = row.copy()
            for i in [1,2,3,4]:  # index of columns to transform
                item = float(row[i])
                if item >= 48:
                    row_transformed[i] = str(int(item - 48))  # Subtract 48 and convert to string
                else:
                    row_transformed[i] = str(int(item))  # Convert to string without subtracting 48
            selected_data.append([row_transformed[i] for i in [1,2,3,4,10]])  # Select the specific columns
        
        with open(output_file_path, 'w') as output_file:
            
            for line in fixed_data:
                output_file.write(line + "\n")
        
            output_file.write("# a b m n rhoa\n")
            for row in selected_data:
                output_file.write("\t".join(row) + "\n")
        
        print(f"Data extraction and conversion completed for {filename}.")

print("All files processed successfully.")

FileNotFoundError: [Errno 2] No such file or directory: 'Raw'